 <p style="padding: 10px;
              color:black;
                font-size:35px;">
            <b>🔘 Guide on handling Outliers</b>
        </p>


<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#FF718F;
           font-size:20px;
           font-family:Nexa;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:black;">
            <b>🔘 Step 1 : Import R Librarie</b>
        </p>
</div>

In [ ]:
#Importer les packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import warnings
warnings.filterwarnings("ignore")

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#FF718F;
           font-size:20px;
           font-family:Nexa;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:black;">
            <b>🔘 Step 1 : Reading Data</b>
        </p>
</div>

In [ ]:
import pandas as pd
df=pd.read_excel("vessel_call_final.xlsx",sheet_name=0)
df

In [ ]:
pd.set_option('display.max_rows',df.shape[0]+1)
pd.set_option('display.max_columns',df.shape[1]+1)


In [ ]:
df.drop('Q.ENTRE',axis='columns', inplace=True)
df.drop('Q.SORTIE',axis='columns', inplace=True)
df.drop('Séjour Quai',axis='columns', inplace=True)
df.drop('OPERATIONS COMMERCIALES',axis='columns', inplace=True)

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')
catvars=[var for var in df if df[var].dtype=='O']



In [ ]:
numvars=[var for var in df.columns if df[var].dtype!='O']
catvars=df.select_dtypes(include=["object"])
numvars=df.select_dtypes(exclude=["object"])


In [ ]:
numvars.info()


In [ ]:
df[catvars].shape

**Valeur abberants**
La valeur aberrante est une terminologie couramment utilisée par les analystes et les scientifiques des données, car elle nécessite une attention particulière, sinon elle peut entraîner des estimations extrêmement erronées. En termes simples, une valeur aberrante est une observation qui semble lointaine et diverge d'un modèle global dans un échantillon.

Prenons un exemple, nous effectuons le profilage des clients et découvrons que le revenu annuel moyen des clients est de 0,8 million de dollars. Mais, il y a deux clients ayant un revenu annuel de 4 et 4,2 millions de dollars. Le revenu annuel de ces deux clients est bien supérieur à celui du reste de la population. Ces deux observations seront considérées comme des valeurs aberrantes.

Les causes des valeurs aberrantes peuvent être classées en deux grandes catégories :

* Artificiel (Erreur) / Non naturel
* Naturel .
* Different type de valeur aberrantes

**Comprenons plus en détail les différents types de valeurs aberrantes :**

Erreurs de saisie de données :Les erreurs humaines telles que les erreurs causées lors de la collecte, de l'enregistrement ou de la saisie des données peuvent entraîner des valeurs aberrantes dans les données. Par exemple : Le revenu annuel d'un client est de 100 000 .Accidentellement,l′opérateurdesaisiemetunzérosupplémentairedanslafigure.Maintenant,lerevenudevient1000000
 , ce qui est 10 fois plus élevé. De toute évidence, ce sera la valeur aberrante par rapport au reste de la population.

**Erreur de mesure :** C'est la source la plus courante de valeurs aberrantes. Cela se produit lorsque l'instrument de mesure utilisé s'avère défectueux. Par exemple : Il y a 10 balances. 9 d'entre eux sont corrects, 1 est défectueux. Le poids mesuré par les personnes sur la machine défectueuse sera supérieur/inférieur à celui du reste des personnes du groupe. Les poids mesurés sur une machine défectueuse peuvent conduire à des valeurs aberrantes.

**Erreur expérimentale :** Une autre cause de valeurs aberrantes est l'erreur expérimentale. Par exemple : dans un sprint de 100 m de 7 coureurs, un coureur n'a pas réussi à se concentrer sur l'appel « Go », ce qui l'a fait partir en retard. Par conséquent, le temps de course du coureur était plus long que celui des autres coureurs. Son temps d'exécution total peut être une valeur aberrante.

**Valeur aberrante intentionnelle :** on la trouve couramment dans les mesures autodéclarées qui impliquent des données sensibles. Par exemple : les adolescents déclarent généralement sous-déclarer la quantité d'alcool qu'ils consomment. Seule une fraction d'entre eux rapporterait la valeur réelle. Ici, les valeurs réelles peuvent ressembler à des valeurs aberrantes, car le reste des adolescents ne déclarent pas la consommation.

**Erreur de traitement des données :** chaque fois que nous effectuons une exploration de données, nous extrayons des données de plusieurs sources. Il est possible que certaines erreurs de manipulation ou d'extraction conduisent à des valeurs aberrantes dans l'ensemble de données.

**Erreur d'échantillonnage :** par exemple, nous devons mesurer la taille des athlètes. Par erreur, nous incluons quelques basketteurs dans l'échantillon. Cette inclusion est susceptible de provoquer des valeurs aberrantes dans l'ensemble de données.

**Valeur aberrante naturelle :** lorsqu'une valeur aberrante n'est pas artificielle (en raison d'une erreur), il s'agit d'une valeur aberrante naturelle. Par exemple : lors de ma dernière mission avec l'une des compagnies d'assurance renommées, j'ai remarqué que la performance des 50 meilleurs conseillers financiers était bien supérieure à celle du reste de la population. Étonnamment, ce n'était pas dû à une erreur. Par conséquent, chaque fois que nous effectuons une activité d'exploration de données avec des conseillers, nous traitions ce segment séparément.

In [ ]:
# view summary statistics in numerical variables

print(round(df[numerical].describe()),2)

**On voit que Rainfall, Evaporation, WindSpeed9am et WindSpeed3pm peut contenir des valeur aberante.
on voit contruire le boxplot**

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#FF718F;
           font-size:20px;
           font-family:Nexa;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:black;">
            <b>How to Find Outliers?</b>
        </p>
</div>

## Z-Core

In [ ]:
import numpy as np
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code
sample_outliers = detect_outliers_zscore(df["DEADWEIGHT"])
print("Outliers from Z-scores method: ", sample_outliers)

## IQR

In [ ]:
import numpy as np  
import seaborn as sns
# jr_boxing_weight_categories
 
Q1=df.DEADWEIGHT.quantile(0.25)
Q2=df.DEADWEIGHT.quantile(0.50)
Q3=df.DEADWEIGHT.quantile(0.75)

IQR = Q3 - Q1
print('Interquartile range is', IQR)
low_lim = Q1 - 1.5 * IQR
up_lim = Q3 + 1.5 * IQR
print('low_limit is', low_lim)
print('up_limit is', up_lim)
outlier =[]
for x in df.DEADWEIGHT:
    if ((x> up_lim) or (x<low_lim)):
         outlier.append(x)
print(' outlier in the dataset is', outlier)

In [ ]:
sns.boxplot(df.DEADWEIGHT)

In [ ]:
# draw boxplots to visualize outliers

plt.figure(figsize=(20,25))
plt.subplot(3,3,1)
sns.boxplot(df['LONGUEUR'])

plt.subplot(3,3,2)
sns.boxplot(df['DEADWEIGHT'])

plt.subplot(3,3,3)
sns.boxplot(df['LARGEUR'])

plt.subplot(3,3,4)
sns.boxplot(df['T.E.AR'])

plt.subplot(3,3,5)
sns.boxplot(df['IMO NUM'])

plt.subplot(3,3,6)
sns.boxplot(df['NET TONNAGE'])

plt.subplot(3,3,7)
sns.boxplot(df['ANNEE DE CONSTRUCTION'])

#plt.subplot(3,3,8)
#sns.boxplot(df['T.E.AV'])

**Les boxplots confirmons l'existance des vleurs aberrantes**

*Quel est l'impact des valeurs aberrantes sur un ensemble de données ?

Les valeurs aberrantes peuvent changer radicalement les résultats de l'analyse des données et de la modélisation statistique. Il existe de nombreux impacts défavorables des valeurs aberrantes dans l'ensemble de données :

Il augmente la variance d'erreur et réduit la puissance des tests statistiques
Si les valeurs aberrantes ne sont pas distribuées de manière aléatoire, elles peuvent diminuer la normalité
Ils peuvent biaiser ou influencer les estimations qui peuvent présenter un intérêt substantiel
Ils peuvent également avoir une incidence sur l'hypothèse de base de la régression, de l'ANOVA et d'autres hypothèses de modèle statistique.

**Vérifier la distribution des variables**
Maintenant, je vais tracer les histogrammes pour vérifier les distributions afin de savoir si elles sont normales ou asymétriques. Si la variable suit une distribution normale, je ferai une analyse des valeurs extrêmes, sinon si elles sont asymétriques, je trouverai l'IQR (gamme interdfle).

In [ ]:
# plot histogram to check distribution

plt.figure(figsize=(25,10))


plt.subplot(3, 3, 1)
fig = df.LONGUEUR.hist(bins=10)
fig.set_xlabel('LONGUEUR')
fig.set_ylabel('LONGUEUR')


plt.subplot(3, 3, 2)
fig = df.DEADWEIGHT.hist(bins=10)
fig.set_xlabel('DEADWEIGHT')
fig.set_ylabel('DEADWEIGHT')


plt.subplot(3, 3, 3)
fig = df.LARGEUR.hist(bins=10)
fig.set_xlabel('LARGEUR')
fig.set_ylabel('LARGEUR')


plt.subplot(3, 3, 4)
fig = df["T_E_AR"].hist(bins=10)
fig.set_xlabel('T.E.AR')
fig.set_ylabel('T.E.AR')

plt.subplot(3, 3, 5)
fig = df["IMO_NUM"].hist(bins=10)
fig.set_xlabel('IMO NUM')
fig.set_ylabel('IMO NUM')

plt.subplot(3, 3, 6)
fig = df["T_E_AV"].hist(bins=10)
fig.set_xlabel('T.E.AV')
fig.set_ylabel('T.E.AV')

plt.subplot(3, 3, 7)
fig = df["NET_TON"].hist(bins=10)
fig.set_xlabel('NET TONNAGE')
fig.set_ylabel('NET TONNAGE')
            
plt.subplot(3, 3, 8)
fig = df["ANNEE_CONST"].hist(bins=10)
fig.set_xlabel('ANNEE DE CONSTRUCTION')
fig.set_ylabel('ANNEE DE CONSTRUCTION')

Nous pouvons voir que les quatre variables sont biaisées. Donc, je vais utiliser la plage interdfle pour trouver les valeurs aberrantes.

In [ ]:
# find outliers for Rainfall variable

IQR = df.LONGUEUR.quantile(0.75) - df.LONGUEUR.quantile(0.25)
Lower_fence = df.LONGUEUR.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.LONGUEUR.quantile(0.75) + (IQR * 1.5)
print('LONGUEUR outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

In [ ]:
# find outliers for Rainfall variable

IQR = df.DEADWEIGHT.quantile(0.75) - df.DEADWEIGHT.quantile(0.25)
Lower_fence = df.DEADWEIGHT.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.DEADWEIGHT.quantile(0.75) + (IQR * 1.5)
print('DEADWEIGHT outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

In [ ]:
# find outliers for Rainfall variable

IQR = df.ANNEE_CONST.quantile(0.75) - df.ANNEE_CONST.quantile(0.25)
Lower_fence = df.ANNEE_CONST.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.ANNEE_CONST.quantile(0.75) + (IQR * 1.5)
print('ANNEE_CONST outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))


In [ ]:
# find outliers for Rainfall variable

IQR = df.IMO_NUM.quantile(0.75) - df.IMO_NUM.quantile(0.25)
Lower_fence = df.IMO_NUM.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.IMO_NUM.quantile(0.75) + (IQR * 1.5)
print('IMO_NUM outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))


In [ ]:
# find outliers for Rainfall variable

IQR = df.NET_TON.quantile(0.75) - df.NET_TON.quantile(0.25)
Lower_fence = df.NET_TON.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.NET_TON.quantile(0.75) + (IQR * 1.5)
print('NET_TON outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))


In [ ]:
# find outliers for Rainfall variable

IQR = df.T_E_AV.quantile(0.75) - df.T_E_AV.quantile(0.25)
Lower_fence = df.T_E_AV.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.T_E_AV.quantile(0.75) + (IQR * 1.5)
print('T_E_AV outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))


In [ ]:
# find outliers for Rainfall variable

IQR = df.T_E_AR.quantile(0.75) - df.T_E_AR.quantile(0.25)
Lower_fence = df.T_E_AR.quantile(0.25) - (IQR * 1.5)
Upper_fence = df.T_E_AR.quantile(0.75) + (IQR * 1.5)
print('T_E_AR outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

In [ ]:
Pour "WindSpeed3pm", les valeurs minimales et maximales sont 0,0 et 87,0. Ainsi, les valeurs aberrantes sont des valeurs > 57.0.

In [ ]:
Correction Valeurs aberrantes dans les variables numériques¶
Nous avons vu que les colonnes Rainfall, Evaporation, WindSpeed9am et WindSpeed3pm contiennent des valeurs aberrantes. J'utiliserai l'approche de codage supérieur pour limiter les valeurs maximales et supprimer les valeurs aberrantes des variables ci-dessus.

In [ ]:
df.columns

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5CCBD0;
           font-size:20px;
           font-family:Nexa;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:black;">
            <b>How to Hnadling Outliers?</b>
        </p>
</div>

<div class="alert alert-block alert-info" style="font-size:14px; font-family:nexa">
   <b>1. Trimming</b> est la méthode permettant de couper ou de supprimer les valeurs aberrantes à la fin de l'ensemble de données. C'est plus facile que toutes les autres méthodes. <br>
   <b>2. Capping</b> définit une limite pour la fonctionnalité et définit la valeur de toutes les valeurs aberrantes dépassant la limite sur la valeur de la limite. Ainsi, dans l'exemple de l'étudiant, nous fixerons une limite de score et modifierons le score de l'étudiant à 2 % par celui-ci. Par exemple, 75 est la limite de score maximum que nous fixons. Le score des 2 % d'élèves aberrants sera fixé à 75. <br>
  <b>3. Percentiles</b> est un centile égal des deux côtés pour détecter les valeurs aberrantes. Une fois que vous avez détecté ces valeurs aberrantes, vous pouvez utiliser soit le découpage, soit le plafonnement pour vous en débarrasser.
</div>

<div class="alert alert-block alert-info" style="font-size:14px; font-family:nexa">
    <strong>Comment gérer les valeurs aberrantes ?</strong>
<br>Jusqu'à présent, nous avons appris à détecter les valeurs aberrantes. La principale question est de savoir comment gérer les valeurs aberrantes ?
<br><br>
Vous trouverez ci-dessous quelques-unes des méthodes permettant de traiter les valeurs aberrantes :
</div>

 #### <p style="color:Blue">Trimming</p>

 #### <p style="color:Blue">Capping with Z-core with normal distriubution variable</p>

In [ ]:
def max_value(df, variable, top):
    return np.where(df[variable]>top, top, df[variable])



df['LONGUEUR'] = max_value(df, 'LONGUEUR', 254.84)
df['LARGEUR'] = max_value(df, 'LARGEUR', 39.55)
df['ANNEE_CONST'] = max_value(df, 'ANNEE_CONST', 2017.5)
df['IMO_NUM'] = max_value(df, 'IMO_NUM', 9963252.25)
df['DEADWEIGHT'] = max_value(df, 'DEADWEIGHT',73644.625)
df['NET_TON'] = max_value(df, 'NET_TON', 27278.0)
df['T_E_AV'] = max_value(df, 'T_E_AV', 13.45)
df['T_E_AR'] = max_value(df, 'T_E_AR', 15.408749999999998)


In [ ]:
def min_value(df, variable, less):
    return np.where(df[variable]<less, less, df[variable])


df['LONGUEUR'] = min_value(df, 'LONGUEUR', 129.91874999999996)
df['LARGEUR'] = min_value(df, 'LARGEUR', 20.11)
df['ANNEE_CONST'] = min_value(df, 'ANNEE_CONST', 1997.5)
df['IMO_NUM'] = min_value(df, 'IMO_NUM', 8841394.25)
df['DEADWEIGHT'] = min_value(df, 'DEADWEIGHT',803.625)
df['NET_TON'] = min_value(df, 'NET_TON', 430)
df['T_E_AV'] = min_value(df, 'T_E_AV', 5.849999999999999)
df['T_E_AR'] = min_value(df, 'T_E_AR', 6.398750000000001)


In [ ]:
df.DEADWEIGHT.max()

 #### <p style="color:Blue">Capping with IQR with Skewesed distriubution variable</p>

In [4]:
def repair_outliers(name):
    ###Finding the IQR
    percentile25 = df[name].quantile(0.25)
    percentile75 = df[name].quantile(0.75)
    
    ### Finding upper and lower limit
    upper_limit = percentile75 + 1.5 * (percentile75-percentile25)
    lower_limit = percentile25 - 1.5 * (percentile75-percentile25)
    
    ### Step-7: Finding Outliers
    #df[df['IMO NUM'] > upper_limit]
    #df[df['IMO NUM'] < lower_limit]

    #### Step-10: Capping
    df[name] = np.where(
    df[name] > upper_limit,upper_limit,
    np.where(df[name] < lower_limit,lower_limit,df[name]
    )
    )


    
repair_outliers('LONGUEUR')
repair_outliers('LARGEUR')
repair_outliers('IMO_NUM')
repair_outliers('ANNEE_CONST')
repair_outliers('T_E_AR')
repair_outliers('T_E_AV')
repair_outliers('NET_TON')
repair_outliers('DEADWEIGHT'

SyntaxError: incomplete input (144438651.py, line 30)

In [ ]:
# draw boxplots to visualize outliers

plt.figure(figsize=(20,25))
plt.subplot(3,3,1)
sns.boxplot(df['LONGUEUR'])

plt.subplot(3,3,2)
sns.boxplot(df['DEADWEIGHT'])

plt.subplot(3,3,3)
sns.boxplot(df['LARGEUR'])

plt.subplot(3,3,4)
sns.boxplot(df['T_E_AR'])

plt.subplot(3,3,5)
sns.boxplot(df['IMO_NUM'])

plt.subplot(3,3,6)
sns.boxplot(df['NET_TON'])

plt.subplot(3,3,7)
sns.boxplot(df['ANNEE_CONST'])

plt.subplot(3,3,8)
sns.boxplot(df['T_E_AV'])

**Quel est l'impact des valeurs aberrantes sur un ensemble de données ?**

* Les valeurs aberrantes peuvent changer radicalement les résultats de l'analyse des données et de la modélisation statistique. Il existe de nombreux impacts défavorables des valeurs aberrantes dans l'ensemble de données :

* Il augmente la variance d'erreur et réduit la puissance des tests statistiques
* Si les valeurs aberrantes ne sont pas distribuées de manière aléatoire, elles peuvent diminuer la normalité
* Ils peuvent biaiser ou influencer les estimations qui peuvent présenter un intérêt substantiel
* Ils peuvent également avoir une incidence sur l'hypothèse de base de la régression, de l'ANOVA et d'autres hypothèses de modèle statistique.

In [ ]:
df[numerical].describe()